Import the necessary libraries and modules.

In [11]:
import pandas as pd
import datetime as dt
import csv
import math

Change the Path variable to point to your CSV file

In [12]:
# C:\Users\joacosta\Work\Beumer\NortFord\Data Analitics\Programs to extract Data\Log Monitor Files\09-18-2025\iteminducted.csv

In [13]:
# path = r"C:\Users\joacosta\Dev\Python\ORF5\Data\iu006rate.csv"
path = r"C:\Users\joacosta\Work\Beumer\NortFord\Data Analitics\Programs to extract Data\Log Monitor Files\09-18-2025\iteminducted.csv"

In [14]:
#  Raw read: semicolon-delimited, no header, don't treat quotes specially
df = pd.read_csv(
    path,
    sep=";",
    header=None,
    engine="python",
    quoting=csv.QUOTE_NONE,      # make " just a character
    skipinitialspace=True,
    on_bad_lines="skip",         # skip malformed rows if any
    dtype=str                    # keep everything as text for now
)

# Trim spaces/quotes around each field
df = df.apply(lambda s: s.str.strip().str.strip('"'))

names = [
    "stamp_raw","flag","system","hostport","module","unused1","unused2","run_hms",
    "seq","mc","code","sq","event","awcsAction","plcRecordNo","itemID","indexNo",
    "awcsStateNow","awcsStateNew","inductionStatus","inductionNo","destinationNo",
    "carrierNo","carrierCount","comHost","comMode","telegramType"
]

df.columns = names[:df.shape[1]]
df.head(10)


,stamp_raw,flag,system,hostport,module,unused1,unused2,run_hms,seq,mc,code,sq
0,250918 094053 136,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:17,147",MC01,54123,SQ 103,"->{event: ""AwcsConverterReceive40ItemInducted""..."
1,250918 094054 068,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:18,091",MC01,54123,SQ 115,"->{event: ""AwcsConverterReceive40ItemInducted""..."
2,250918 094055 238,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:19,262",MC01,54123,SQ 128,"->{event: ""AwcsConverterReceive40ItemInducted""..."
3,250918 094056 950,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:20,978",MC01,54123,SQ 141,"->{event: ""AwcsConverterReceive40ItemInducted""..."
4,250918 094057 976,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:21,953",MC01,54123,SQ 155,"->{event: ""AwcsConverterReceive40ItemInducted""..."
5,250918 094059 359,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:23,388",MC01,54123,SQ 172,"->{event: ""AwcsConverterReceive40ItemInducted""..."
6,250918 094100 333,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:24,332",MC01,54123,SQ 181,"->{event: ""AwcsConverterReceive40ItemInducted""..."
7,250918 094101 521,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:25,528",MC01,54123,SQ 197,"->{event: ""AwcsConverterReceive40ItemInducted""..."
8,250918 094102 713,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:26,742",MC01,54123,SQ 209,"->{event: ""AwcsConverterReceive40ItemInducted""..."
9,250918 094103 674,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:27,686",MC01,54123,SQ 218,"->{event: ""AwcsConverterReceive40ItemInducted""..."


Adding timestamp Column in a legible format

In [15]:
sr = df["stamp_raw"].str.replace(" ", "", regex=False)  # e.g. 250917160001278
df["timestamp"] = pd.to_datetime(sr, format="%d%m%y%H%M%S%f", errors="coerce")

df.head()


,stamp_raw,flag,system,hostport,module,unused1,unused2,run_hms,seq,mc,code,sq,timestamp
0,250918 094053 136,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:17,147",MC01,54123,SQ 103,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 09:40:53.136
1,250918 094054 068,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:18,091",MC01,54123,SQ 115,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 09:40:54.068
2,250918 094055 238,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:19,262",MC01,54123,SQ 128,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 09:40:55.238
3,250918 094056 950,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:20,978",MC01,54123,SQ 141,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 09:40:56.950
4,250918 094057 976,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"21:22:21,953",MC01,54123,SQ 155,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 09:40:57.976


Getting the Start time and End time of the data and calculating the delta of time plus how many records we have in this dataset

In [16]:
valid = df["timestamp"].notna()
start = df.loc[valid, "timestamp"].min()
end   = df.loc[valid, "timestamp"].max()
delta = end - start

print(f"Start : {start}")
print(f"End   : {end}")
print(f"Delta : {delta}  (~{delta.total_seconds()/60:.2f} minutes)")
print(f"Rows  : {valid.sum()} valid / {len(df)-valid.sum()} invalid")

Start : 2018-09-25 09:40:53.136000
End   : 2018-09-25 10:41:29.771000
Delta : 0 days 01:00:36.635000  (~60.61 minutes)
Rows  : 2564 valid / 0 invalid


In [17]:
# data limits from your parsed timestamps
data_min = df["timestamp"].min()
data_max = df["timestamp"].max()

def select_window(df, start_input, minutes=10):
    base_date = data_min.normalize()                # date of your dataset
    ts_in = pd.to_datetime(start_input)             # e.g. "16:40:00" or "4:40 PM"
    start = base_date + (ts_in - ts_in.normalize()) # anchor time-of-day to base_date
    end   = start + pd.Timedelta(minutes=minutes)

    print(f"Data range : {data_min} → {data_max}")
    print(f"Requested  : {start} → {end}  ({minutes} min)")
    print("Start OK   :", data_min <= start <= data_max)
    if end > data_max:
        print("Note       : End is beyond data_max; results will stop at data_max.")

    mask = (df["timestamp"] >= start) & (df["timestamp"] <= end)
    return df.loc[mask].copy(), start, end

# EXAMPLE:
window_df, start_ts, end_ts = select_window(df, "16:06:00", minutes=10)
window_df.head(10)

Data range : 2018-09-25 09:40:53.136000 → 2018-09-25 10:41:29.771000
Requested  : 2018-09-25 16:06:00 → 2018-09-25 16:16:00  (10 min)
Start OK   : False
Note       : End is beyond data_max; results will stop at data_max.


,stamp_raw,flag,system,hostport,module,unused1,unused2,run_hms,seq,mc,code,sq,timestamp


Asking to the user what time window wants to analyze.

In general we aim for the best 10 minutes in the production state.

In [18]:
data_min = df["timestamp"].min()
base_date = data_min.normalize()

def select_window_cli_24h(df):
    s = input("Start time 24h (e.g. 16:00 or 16:00:30 or 16): ").strip()
    m = input("Window minutes [10]: ").strip()
    minutes = int(m) if m else 10
    try:
        t = dt.time.fromisoformat(s)
    except ValueError:
        t = dt.time(int(s), 0, 0)
    start = pd.Timestamp(dt.datetime.combine(base_date.date(), t))
    end   = start + pd.Timedelta(minutes=minutes)
    mask  = (df["timestamp"] >= start) & (df["timestamp"] <= end)
    win   = df.loc[mask].copy()
    print(f"Window: {start} → {end}  ({minutes} min) | Rows: {len(win)}")
    return win, start, end

# use it:
window_df, start_ts, end_ts = select_window_cli_24h(df)
window_df.head(10)


Window: 2018-09-25 10:25:00 → 2018-09-25 10:35:00  (10 min) | Rows: 451


,stamp_raw,flag,system,hostport,module,unused1,unused2,run_hms,seq,mc,code,sq,timestamp
1941,250918 102501 438,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"22:06:25,409",MC01,54123,SQ 203,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 10:25:01.438
1942,250918 102502 842,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"22:06:26,833",MC01,54123,SQ 226,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 10:25:02.842
1943,250918 102504 001,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"22:06:28,010",MC01,54123,SQ 248,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 10:25:04.001
1944,250918 102504 951,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"22:06:28,955",MC01,54123,SQ 000,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 10:25:04.951
1945,250918 102505 936,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"22:06:29,949",MC01,54123,SQ 019,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 10:25:05.936
1946,250918 102508 125,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"22:06:32,131",MC01,54123,SQ 052,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 10:25:08.125
1947,250918 102509 783,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"22:06:33,755",MC01,54123,SQ 083,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 10:25:09.783
1948,250918 102510 744,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"22:06:34,750",MC01,54123,SQ 099,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 10:25:10.744
1949,250918 102511 697,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"22:06:35,703",MC01,54123,SQ 121,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 10:25:11.697
1950,250918 102513 122,N,SMC,10.158.244.100:7200,AWCS.Comm,,,"22:06:37,129",MC01,54123,SQ 144,"->{event: ""AwcsConverterReceive40ItemInducted""...",2018-09-25 10:25:13.122


Calculating metrics as Items per second, Average time between items, Average time per item

In [19]:
n_items = len(window_df)
delta_secs = delta.total_seconds()
delta_min = delta_secs / 60.0

sec_per_bag   = (delta_secs / n_items) if n_items > 0 else float("nan")
items_per_sec = (n_items / delta_secs) if delta_secs > 0 else float("nan")
items_per_min = items_per_sec * 60 if delta_secs > 0 else float("nan")

print(f"Items in window_df : {n_items}")
print(f"Overall delta (s)  : {delta_secs:.2f}")
print(f"Overall delta (min): {delta_min:.2f}")
print(f"Sec per bag        : {sec_per_bag:.3f} s/bag")
print(f"Throughput         : {items_per_sec:.4f} items/s  ({items_per_min:.2f} items/min)")


Items in window_df : 451
Overall delta (s)  : 3636.64
Overall delta (min): 60.61
Sec per bag        : 8.063 s/bag
Throughput         : 0.1240 items/s  (7.44 items/min)


In [20]:
TARGET_PPH = 2820  # pieces per hour target

n_items = len(window_df)

# PPH using overall dataset duration (delta from min→max)
delta_secs = delta.total_seconds()
pph_overall = n_items * 3600 / delta_secs if delta_secs > 0 else float("nan")

# PPH using the selected window duration
window_secs = (end_ts - start_ts).total_seconds()
pph_window = n_items * 3600 / window_secs if window_secs > 0 else float("nan")

print(f"Items in window_df     : {n_items}")
print(f"PPH (overall delta)    : {pph_overall:.2f}")
print(f"PPH (window {window_secs/60:.1f} min): {pph_window:.2f}")
print(f"Target PPH             : {TARGET_PPH}")
print(f"Attainment vs target (overall): {pph_overall/TARGET_PPH*100:.1f}%")
print(f"Attainment vs target (window) : {pph_window/TARGET_PPH*100:.1f}%")


Items in window_df     : 451
PPH (overall delta)    : 446.46
PPH (window 10.0 min): 2706.00
Target PPH             : 2820
Attainment vs target (overall): 15.8%
Attainment vs target (window) : 96.0%


In [21]:
# --- Counts & durations ---
n_items = len(window_df)
window_secs = (end_ts - start_ts).total_seconds()

# Items/hour in this window
pph = n_items * 3600 / window_secs if window_secs > 0 else float("nan")

# Pass/fail (pass if result >= target)
passed = (pph >= TARGET_PPH) if window_secs > 0 and n_items > 0 else False

# How many items would be needed to pass in this same window?
needed_items_for_pass = math.ceil(TARGET_PPH * window_secs / 3600) if window_secs > 0 else float("nan")
items_short = max(0, needed_items_for_pass - n_items) if window_secs > 0 else float("nan")

# Seconds per bag (current vs target) — handy context
spb_current = window_secs / n_items if n_items > 0 else float("inf")
spb_target  = 3600 / TARGET_PPH

# Recommendation text
if window_secs > 0 and n_items > 0 and items_short > 0:
    recommendation = (
        f"Need +{items_short} items in this {window_secs/60:.1f}-min window "
        f"or reduce sec/bag to ≤ {spb_target:.2f} (current {spb_current:.2f})."
    )
else:
    recommendation = "On target or above. Maintain current rate."

# --- Summary line pieces ---
line_name = "IU006"        # <- set this as needed
measuring_point = "MP1000605"  # <- set this as needed
meridiem = end_ts.strftime('%p')                  # 'AM' or 'PM' based on end time
time_label = f"{start_ts:%H%M}-{end_ts:%H%M}{meridiem}"
summary_line = f"{line_name};Measuring Point: {measuring_point}: Time {time_label}; Total Inducted = {n_items}"

# Quick printout
print(f"PPH (window): {pph:.2f} | Target: {TARGET_PPH} | Passed: {passed}")
print(f"Items in window: {n_items} | Items short to pass: {items_short}")
print(f"Recommendation: {recommendation}")
print(summary_line)


PPH (window): 2706.00 | Target: 2820 | Passed: False
Items in window: 451 | Items short to pass: 19
Recommendation: Need +19 items in this 10.0-min window or reduce sec/bag to ≤ 1.28 (current 1.33).
IU006;Measuring Point: MP1000605: Time 1025-1035AM; Total Inducted = 451
